# library

In [51]:
import shutil

#to delete a folder and its contents
def delete_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)

delete_folder("/content/testing")
delete_folder("/content/training")
delete_folder("/content/validation")



In [ ]:
! pip install pytube

In [ ]:
import cv2
import os
from moviepy.editor import VideoFileClip
from pytube import YouTube

# Youtube video download

In [ ]:
youtube_link = "https://www.youtube.com/watch?v=E_k3p16YLe8"
output_path = "/content"
video_filename = download_youtube_video(youtube_link, output_path)

In [ ]:
def download_youtube_video(youtube_link, output_path):
    yt = YouTube(youtube_link)
    video = yt.streams.get_highest_resolution()
    video.download(output_path)
    return video.default_filename


In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip

video_path = "/content/Video Recognition Detecting People People Talking on Phone (deep learning).mp4"
output_path = "trimmed_video.mp4"

video_clip = VideoFileClip(video_path)
trimmed_clip = video_clip.subclip(0, 60)
trimmed_clip.write_videofile(output_path, codec="libx264", fps=video_clip.fps)
video_clip.close()


t:   0%|          | 2/1799 [04:03<60:49:48, 121.86s/it, now=None]

Moviepy - Building video trimmed_video.mp4.
MoviePy - Writing audio in trimmed_videoTEMP_MPY_wvf_snd.mp3



t:   0%|          | 2/1799 [04:06<61:25:24, 123.05s/it, now=None]

MoviePy - Done.
Moviepy - Writing video trimmed_video.mp4




t:   0%|          | 2/1799 [06:17<94:15:36, 188.83s/it, now=None]

Moviepy - Done !
Moviepy - video ready trimmed_video.mp4


# Dataset Preparation

In [54]:
# Function to extract frames from video
def extract_frames(video_path, output_folder, fps):
    cap = cv2.VideoCapture(video_path)
    success, image = cap.read()
    count = 0
    while success:
        cv2.imwrite(os.path.join(output_folder, f"frame_{count}.jpg"), image)
        count += 1
        cap.set(cv2.CAP_PROP_POS_FRAMES, count * fps)
        success, image = cap.read()
    cap.release()

video_path = "/content/trimmed_video.mp4"
output_folder = "images"

os.makedirs(output_folder, exist_ok=True)

# Extract frames from video at 40fps
extract_frames(video_path, output_folder, fps=40)

print("Frames extracted successfully.")


Frames extracted successfully.


In [55]:
from sklearn.model_selection import train_test_split

image_files = [f for f in os.listdir(output_folder) if f.endswith('.jpg')]

train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)
train_files, val_files = train_test_split(train_files, test_size=0.1, random_state=42)

print("Data split into training, validation, and testing sets.")

Data split into training, validation, and testing sets.


In [56]:
import pandas as pd

# Define directories for training, validation, and testing
os.makedirs("training", exist_ok=True)
os.makedirs("validation", exist_ok=True)
os.makedirs("testing", exist_ok=True)

# Save file paths to CSV for each set
def save_to_csv(file_paths, csv_filename):
    labels = [{"filename": f, "label": 0} for f in file_paths]
    df = pd.DataFrame(labels)
    df.to_csv(csv_filename, index=False)

save_to_csv(train_files, "training/training_labels.csv")
save_to_csv(val_files, "validation/validation_labels.csv")
save_to_csv(test_files, "testing/testing_labels.csv")

def move_files(file_paths, destination_folder):
    for f in file_paths:
        os.replace(os.path.join(output_folder, f), os.path.join(destination_folder, f))

move_files(train_files, "training")
move_files(val_files, "validation")
move_files(test_files, "testing")

print("Data preparation completed.")


Data preparation completed.


In [57]:
import zipfile

# Function to create a zip file from a directory
def create_zip(directory, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(directory):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), directory))

# Paths to directories
training_folder = 'training'
validation_folder = 'validation'
testing_folder = 'testing'

# Create zip files
create_zip(training_folder, 'training.zip')
create_zip(validation_folder, 'validation.zip')
create_zip(testing_folder, 'testing.zip')

print("Zip files created successfully.")

shutil.move('training.zip', '/content/training.zip')
shutil.move('validation.zip', '/content/validation.zip')
shutil.move('testing.zip', '/content/testing.zip')

print("Zip files moved to /content directory.")


Zip files created successfully.
Zip files moved to /content directory.


In [58]:
from google.colab import files
files.download('/content/training.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>